In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


In [2]:


file = r'C:\Users\hyunj\Seoul_Strolling_Adventure\대중교통위치\my_dataframe5.xlsx'


review = r"C:\Users\hyunj\Seoul_Strolling_Adventure\popularity\zb_review_data_final.csv"  # 입력 데이터
popularity = r"C:\Users\hyunj\Seoul_Strolling_Adventure\popularity\popularity_result.csv"    # 결과(출력) 데이터


In [3]:
# 1. 데이터 로드 (my_dataframe5.xlsx)
df_region = pd.read_excel(file)
df_review = pd.read_csv(review, encoding='utf-8-sig')
df_popularity = pd.read_csv(popularity, encoding='utf-8-sig')
# df_popularity.head(1)  # 데이터 확인

In [4]:

# NaN 값을 'cat3'로 채우기
# NaN 값을 cat3 열의 해당 값으로 채우기
df_review['리뷰내용'] = df_review.apply(
    lambda row: row['cat3'] if pd.isna(row['리뷰내용']) else row['리뷰내용'],
    axis=1
)


In [5]:
import re, numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1️⃣ 불용어를 'list' 로 선언
STOPWORDS = [
    "맛", "메뉴", "가격", "주문", "서비스", "정말",
    "진짜", "너무", "그리고", "음식", "여기", "그냥"
]

# 2️⃣ 토큰 패턴
TOKEN_PATTERN = r"(?u)\b[가-힣A-Za-z0-9]{2,}\b"

# 3️⃣ 새 Vectorizer 생성  (이전 vectorizer 변수는 사용 X)
vectorizer = TfidfVectorizer(
    token_pattern=TOKEN_PATTERN,
    stop_words=STOPWORDS,     # ← 반드시 'list'
    lowercase=False
)

# 4️⃣ 리뷰 TF-IDF 계산
tfidf = vectorizer.fit_transform(df_review["리뷰내용"].fillna(""))

# 5️⃣ 키워드 추출 함수
feature_names = np.array(vectorizer.get_feature_names_out())

def top_keywords(row_vec, top_n=5):
    if row_vec.nnz == 0:
        return []
    idx = row_vec.indices[row_vec.data.argsort()[::-1]][:top_n]
    return feature_names[idx].tolist()

df_review["keywords"] = [
    top_keywords(tfidf[i]) for i in range(tfidf.shape[0])
]

In [6]:
df_review.head(5)  # 데이터 확인

,검색어,장소명,평점,주소,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,...,연도월,cat1,cat2,cat3,카테고리,행정동,감정레이블,감정확률,긍정점수,keywords
0,가나아트파크,가나아트파크,3.2,경기도 양주시 장흥면 권율로 117,짜다짜,3.0,2025.03.03.,이 가격에 이 시설이라면 리모델링이 많이 필요함,가나아트파크,경기도 양주시 장흥면 권율로 117,...,2025-03,인문(문화/예술/역사),휴양관광지,테마공원,힐링,장흥면,1.0,0.687093,4.24,"[리모델링이, 시설이라면, 필요함, 가격에, 많이]"
1,가나아트파크,가나아트파크,3.2,경기도 양주시 장흥면 권율로 117,리커버리,4.0,2025.02.15.,적당~~~합니다. 입장료 9000원 정도가 적절. 입장료는 11000원. 일단 넓은...,가나아트파크,경기도 양주시 장흥면 권율로 117,...,2025-02,인문(문화/예술/역사),휴양관광지,테마공원,힐링,장흥면,2.0,0.732736,7.51,"[적절, 넓은건, 11000원, 9000원, 적당]"
2,가나아트파크,가나아트파크,3.2,경기도 양주시 장흥면 권율로 117,BC😊,4.0,2024.06.02.,"생각보다 아주 넓진 않지만, 사람이 붐비진 않아서 피크닉으로 한번쯤 가볼만합니다~~",가나아트파크,경기도 양주시 장흥면 권율로 117,...,2024-06,인문(문화/예술/역사),휴양관광지,테마공원,힐링,장흥면,1.0,0.877998,4.83,"[붐비진, 피크닉으로, 넓진, 가볼만합니다, 한번쯤]"
3,가나아트파크,가나아트파크,3.2,경기도 양주시 장흥면 권율로 117,포뇨v,4.0,2023.06.05.,예매하고 가야 싸요~! 물놀이 최고👍🏻,가나아트파크,경기도 양주시 장흥면 권율로 117,...,2023-06,인문(문화/예술/역사),휴양관광지,테마공원,힐링,장흥면,2.0,0.986721,9.68,"[예매하고, 싸요, 물놀이, 가야, 최고]"
4,가나아트파크,가나아트파크,3.2,경기도 양주시 장흥면 권율로 117,sorasun,3.0,2023.04.24.,테마공원,가나아트파크,경기도 양주시 장흥면 권율로 117,...,2023-04,인문(문화/예술/역사),휴양관광지,테마공원,힐링,장흥면,NaN,NaN,6.00,[테마공원]


In [7]:
key_cols = ["검색어", "장소명", "주소"]

def merge_keywords(series):
    merged, seen = [], set()
    for lst in series:
        for kw in lst:
            if kw not in seen:
                seen.add(kw)
                merged.append(kw)
    return merged

# ── 1) 컬럼별 집계 함수 자동 생성 ─────────────────────
agg_dict = {}

for col, dtype in df_review.dtypes.items():
    if col in key_cols:
        continue                           # 그룹 키는 제외
    if col == "keywords":
        agg_dict[col] = merge_keywords
    elif np.issubdtype(dtype, np.number):  # 숫자 → 평균
        agg_dict[col] = "mean"
    else:                                  # 그 외(문자열·날짜) → 첫 행
        agg_dict[col] = "first"

# ── 2) 그룹화 & 집계 ───────────────────────────────────
df_review = (
    df_review
    .groupby(key_cols, as_index=False)
    .agg(agg_dict)
    .round(2)      # 숫자 평균값 2자리로 보기 좋게
)

df_review.head(3)

,검색어,장소명,주소,평점,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,...,연도월,cat1,cat2,cat3,카테고리,행정동,감정레이블,감정확률,긍정점수,keywords
0,(구)인천일본제58은행지점,구 일본제58은행 인천지점,인천광역시 중구 신포로23번길 69-1,3.5,Traveler,3.50,2025.01.18.,"전시관이라고 생각하고 갔지만 실내에 가볼 수 없어 아쉬웠다. 제1은행, 제18은행과...",(구)인천일본제58은행지점,인천광역시 중구 신포로23번길 69-1 중구음식업지부,...,2025-01,인문(문화/예술/역사),역사관광지,유적지/사적지,문화,신포동,1.50,0.69,5.72,"[제1은행, 제18은행과는, 보존해서, 해놓았으면, 전시관이라고, 제58은행, 인천..."
1,(유)헤세의정원,헤세의정원,경기도 양주시 장흥면 호국로550번길 111,3.5,효자,3.00,2024.08.10.,피자 도우도 직접 만들지 않고 기존 제품 사용샐러드 소스도 깡통 따서 만든 기성제품...,(유)헤세의정원,경기도 양주시 장흥면 호국로550번길 111,...,2024-08,음식,음식점,서양식,음식,장흥면,1.25,0.80,6.05,"[실망안가련다, 느낌정말, 사용샐러드, 기성제품, 깡통, 대형견과, 분리도, 강아지..."
2,007 양평점,007 양평점,경기도 양평군 양평읍 물안개공원길 5-1 1-2층,3.6,지니,3.59,2025.04.07.,피자 먹었을 때 속이 더부룩하지 않고 편해서 좋았습니다.,007 양평점,경기도 양평군 양평읍 물안개공원길 5-1,...,2025-04,음식,음식점,카페/전통찻집,음식,양평읍,1.37,0.82,6.74,"[더부룩하지, 먹었을, 편해서, 속이, 피자, 내가, 던킨처럼, 걱정을겁나, 극찬했..."


In [8]:

df_region = pd.merge(df_region, df_review[['addr1', '긍정점수']], how='left', on='addr1')
df_region = pd.merge(df_region, df_review[['addr1', 'keywords']], how='left', on='addr1')
df_region = df_region.drop_duplicates(subset=['title'], keep='first').reset_index(drop=True)
df_region.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,긍정점수,keywords
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000013,08호선,NaN,NaN,NaN,푼주,8.52,"[화내냐, 똑디, 쓸거면, 체험단, 고생해서, 기분이예요집에서, 생일날, 대접을, ..."
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,GGB103000005,02호선,NaN,NaN,NaN,레이지 유스,7.59,"[조을듯합니다, 켜주시면, 느좋카, 보내구, 약간만, 사장님이, 친절하시고, 좋아요]"


In [9]:
df = df_region

In [10]:
import pandas as pd
from math import radians, sin, cos, sqrt, asin

# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

# 1) 음식 카테고리 필터링
df_food = df[df['cat1'] == '자연'].copy()

# 2) 서울역 좌표
station_lat, station_lon = 37.5561, 126.9726

# 3) 각 음식점까지 거리 계산
df_food['distance'] = df_food.apply(
    lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
)

# 4) 가장 가까운 10개 음식점 추출
top10_closest = df_food.nsmallest(10, 'distance').reset_index(drop=True)

# 5) 긍정점수 기준으로 정렬
top10_sorted = top10_closest.sort_values(by='긍정점수', ascending=False).reset_index(drop=True)

# 키워드 열 통합
top10_sorted = pd.merge(
    top10_sorted,
    df_review[['addr1', 'keywords']],
    how='left',
    on='addr1'
)

# 중복 열 이름 정리: keywords_y를 keywords로 변경
top10_sorted['keywords'] = top10_sorted['keywords_y']

# 불필요한 열 제거
top10_sorted = top10_sorted.drop(columns=['keywords_x', 'keywords_y'])

top10_sorted.head(3)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,긍정점수,distance,keywords
0,서울특별시 서대문구 봉원동,자연,자연관광지,산,126.944774,37.577236,안산초등학교(중),무악재역,안산초등학교(중),GGB112000404,03호선,NaN,NaN,NaN,서대문안산,9.08,3.396816,"[폭포에서, 40분이면, 산계단, 쉬워질듯, 시작해, 메타세콰이어숲, 홍제폭포까지,..."
1,서울특별시 서대문구 봉원동,자연,자연관광지,산,126.944774,37.577236,안산초등학교(중),무악재역,안산초등학교(중),GGB112000404,03호선,NaN,NaN,NaN,서대문안산,9.08,3.396816,"[저질체력이라, 한강도, 끝내줬어요, 시내가, 훤히, 379번째, 등산, 쉽다고, ..."
2,서울특별시 종로구 인왕산로1길 29,자연,자연관광지,산,126.964083,37.573567,독립문고가차도(경유),독립문역,독립문고가차도(경유),GGB277102320,03호선,NaN,NaN,NaN,인왕산,8.61,2.082268,"[힘들지만, 좋았습니다, 사극에서, 산이죠, 궁궐이, 언급되는, 날좋을때, 고도에,..."


In [11]:

stage_names = top10_sorted["title"].tolist()

In [12]:
def remove_duplicates_preserve_order(input_list):
    seen = set()
    return [x for x in input_list if not (x in seen or seen.add(x))]

In [13]:
# 중복 제거
stage_names = remove_duplicates_preserve_order(stage_names)

In [14]:
stage_names

['서대문안산',
 '인왕산',
 '창경궁대온실',
 '창경궁 명정전',
 '창경궁 자생식물학습장',
 '창경궁 홍화문',
 '수성동계곡',
 '남산순환나들길',
 '남산인권숲',
 '선바위(선암)']

In [15]:
top10_closest = df_food.nsmallest(10, 'distance')  # 인덱스 유지
top10_sorted = top10_closest.sort_values(by='긍정점수', ascending=False)  # 인덱스 유지


In [16]:
from sklearn.neighbors import NearestNeighbors

df_candidates = df_food.drop(top10_sorted.index)
coords_rad = np.radians(df_candidates[['mapy','mapx']].values)
EARTH_RADIUS = 6371.0


In [17]:
print("stage_names 길이:", len(stage_names))
print("top10_sorted 길이:", len(top10_sorted))

stage_names 길이: 10
top10_sorted 길이: 10


In [18]:
# # stage_names 동기화
# stage_names = top10_sorted["title"].tolist()
# if len(stage_names) != len(top10_sorted):
#     raise ValueError("stage_names와 top10_sorted 길이가 다릅니다.")


In [19]:
# 4) 반경 10km NN 모델 (haversine)
nn = NearestNeighbors(radius=10/EARTH_RADIUS, metric='haversine')
nn.fit(coords_rad)

used_idxs = set()
routes = []

for idx, (index, row) in enumerate(top10_sorted.iterrows()):
    stage = stage_names[idx]

    pt1_rad = np.radians([[row['mapy'], row['mapx']]])

    # — 1차 추천: 5~10km 필터 후 score↓, dist↑ —
    d1_rad, i1 = nn.radius_neighbors(pt1_rad, return_distance=True)
    d1_km = d1_rad[0] * EARTH_RADIUS

    # 5km 이상만
    mask1 = d1_km >= 5
    cand1_all = i1[0][mask1]
    dist1_all = d1_km[mask1]

    # 사용 안 한 것만 모아서 DataFrame
    pairs1 = [(i, dist) for i, dist in zip(cand1_all, dist1_all) if i not in used_idxs]
    tmp1 = pd.DataFrame(pairs1, columns=['idx','dist'])
    # iloc으로 긍정점수 불러오기
    tmp1['긍정점수'] = df_candidates.iloc[tmp1['idx']]['긍정점수'].values
    # score 내림차순, 거리 오름차순
    tmp1 = tmp1.sort_values(['긍정점수','dist'], ascending=[False,True])

    rec1_idxs = tmp1['idx'].tolist()[:5]
    used_idxs.update(rec1_idxs)

    rec1_df   = df_candidates.iloc[rec1_idxs]
    rec1_list = rec1_df['title'].tolist()
    rec1_first= rec1_df.iloc[0] if not rec1_df.empty else None

    # — 2차 추천 (rec1_first 기준 동일 방식) —
    rec2_idxs = []
    if rec1_first is not None:
        pt2_rad = np.radians([[rec1_first['mapy'], rec1_first['mapx']]])
        d2_rad, i2 = nn.radius_neighbors(pt2_rad, return_distance=True)
        d2_km = d2_rad[0] * EARTH_RADIUS

        mask2 = d2_km >= 5
        cand2_all = i2[0][mask2]
        dist2_all = d2_km[mask2]

        pairs2 = [(i, dist) for i, dist in zip(cand2_all, dist2_all) if i not in used_idxs]
        tmp2 = pd.DataFrame(pairs2, columns=['idx','dist'])
        tmp2['긍정점수'] = df_candidates.iloc[tmp2['idx']]['긍정점수'].values
        tmp2 = tmp2.sort_values(['긍정점수','dist'], ascending=[False,True])

        rec2_idxs = tmp2['idx'].tolist()[:5]
        used_idxs.update(rec2_idxs)

    rec2_df    = df_candidates.iloc[rec2_idxs]
    rec2_list  = rec2_df['title'].tolist()
    rec2_first = rec2_df.iloc[0] if not rec2_df.empty else None

    # — 이동수단1 / 내리는곳1 결정 —
    if idx % 2 == 0:
        mode1, start1 = '지하철', row['closest_subway_station']
    else:
        mode1, start1 = '버스', row['matched_station_name']

    if rec1_first is not None:
        end1 = (rec1_first['closest_subway_station']
                if mode1=='지하철'
                else rec1_first['matched_station_name'])
    else:
        end1 = None

    # — 이동수단2 / 타는곳2, 내리는곳2 결정 —
    mode2 = '버스' if mode1=='지하철' else '지하철'
    if rec1_first is not None:
        start2 = (rec1_first['closest_subway_station']
                  if mode2=='지하철'
                  else rec1_first['matched_station_name'])
    else:
        start2 = None
    if rec2_first is not None:
        end2 = (rec2_first['closest_subway_station']
                if mode2=='지하철'
                else rec2_first['matched_station_name'])
    else:
        end2 = None

    routes.append({
        '단계': stage,
        '이동수단1': mode1,
        '타는곳1': start1,
        '내리는곳1': end1,
        '추천음식점1': ', '.join(rec1_list) or '없음',
        '이동수단2': mode2,
        '타는곳2': start2,
        '내리는곳2': end2,
        '추천음식점2': ', '.join(rec2_list) or '없음'
    })

result_df = pd.DataFrame(routes)
result_df 

,단계,이동수단1,타는곳1,내리는곳1,추천음식점1,이동수단2,타는곳2,내리는곳2,추천음식점2
0,서대문안산,지하철,무악재역,북한산보국문역,"북한산국립공원(서울), 월드컵공원 유아숲체험원, 난지천공원, 안양천제방벚꽃길, 홍릉숲",버스,국민대학교앞,상계동주유소,"수락산 당고개지구 공원, 무악산 동봉수대 터, 봉화산(서울), 홍제천인공폭포, 벽운계곡"
1,인왕산,버스,독립문고가차도(경유),방학동신동아타워,"솔밭근린공원, 한강, 불광천, 진관사계곡, 진관사(서울)",지하철,솔밭공원역,수락산역,"불암산, 서울창포원, 백악마루, 송추계곡, 일영허브랜드(숲길정원)"
2,창경궁대온실,지하철,혜화역,용마산역,"용마산, 초안산, 아차산 어울림정원, 여의도샛강생태공원, 긴고랑계곡",버스,용마폭포공원,구산문화마을.구산성당,"하남 나무고아원, 오동근린공원 유아숲체험장, 오동근린공원, 길동생태공원, 일자산자연공원"
3,창경궁 명정전,버스,창경궁.서울대학교병원(중),서빙고역1번출구,"용산공원 부분개방부지, 거북선나루터, 홍제폭포, 녹번서공원, 삼각산",지하철,서빙고역,청계산입구역,"여의천, 서울대공원산림욕장, 산기슭 공원, 대모산숲속야생화원, 일원에코파크"
4,창경궁 자생식물학습장,지하철,혜화역,솔밭공원역,"북한산 백운대(우이동), 아차산, 아차산 고구려정, 망우산(서울)",버스,방학동신동아타워,권율장군묘,"송암스페이스센터, 백사실계곡, 수락산, 산들소리"
5,창경궁 홍화문,버스,창경궁.서울대학교병원(중),None,없음,지하철,None,None,없음
6,수성동계곡,지하철,독립문역,염창역,150년 수령 느티나무,버스,목동3단지,푸른수목원,"푸른수목원, 무릉도원수목원, 부천자연생태박물관, 진달래동산, 도덕산공원"
7,남산순환나들길,버스,남산3호터널,우면산터널TG(경유),"우면산자연생태공원, 신도림테크노근린공원",지하철,남부터미널역,정부과천청사역,"과천야생화자연학습장, 마루공원, 과천 은행나무길, 청계산맑은숲공원, 백운호수"
8,남산인권숲,지하철,명동역,None,없음,버스,None,None,없음
9,선바위(선암),버스,독립문역.한성과학고(중),None,없음,지하철,None,None,없음


In [20]:
# 모든 추천음식점의 키워드를 모아 새로운 열 생성
def collect_keywords(row, df_candidates):
    # '단계', '추천음식점1', '추천음식점2'의 키워드를 추출
    all_titles = row['단계'].split(", ") + row['추천음식점1'].split(", ") + row['추천음식점2'].split(", ")
    keyword_set = set()  # 중복 제거를 위한 집합
    for title in all_titles:
        keywords = df_candidates[df_candidates['title'] == title]['keywords']
        if not keywords.empty:
            # 키워드 값이 리스트인지 확인
            if isinstance(keywords.iloc[0], list):
                keyword_set.update(keywords.iloc[0])  # 키워드를 집합에 추가
    return list(keyword_set) if keyword_set else ["No keywords"]  # 기본값 처리

# 단계별 데이터 처리
result_df['모든추천장소키워드'] = result_df.apply(lambda row: collect_keywords(row, df_candidates), axis=1)

In [21]:
result_df.insert(0, '모든추천장소키워드', result_df.pop('모든추천장소키워드'))  # '모든추천장소키워드'를 맨 앞에 추가

result_df.rename(columns={'추천음식점2': '추천장소3'}, inplace=True)  # 추천음식점2를 추천음식점3으로 변경
result_df.rename(columns={'추천음식점1': '추천장소2'}, inplace=True) 
result_df.rename(columns={'단계': '추천장소점1'}, inplace=True) 


In [22]:
result_df

,모든추천장소키워드,추천장소점1,이동수단1,타는곳1,내리는곳1,추천장소2,이동수단2,타는곳2,내리는곳2,추천장소3
0,"[운동할수, 서울숲과는, 정비되어, 지나가는, 평화롭고, 좋다, 편합니다, 걷기만,...",서대문안산,지하철,무악재역,북한산보국문역,"북한산국립공원(서울), 월드컵공원 유아숲체험원, 난지천공원, 안양천제방벚꽃길, 홍릉숲",버스,국민대학교앞,상계동주유소,"수락산 당고개지구 공원, 무악산 동봉수대 터, 봉화산(서울), 홍제천인공폭포, 벽운계곡"
1,"[안좋고장난감들이, 사찰이, 생겨, 자전거, 작가의, 국립공원, 센터가, 방문했는데...",인왕산,버스,독립문고가차도(경유),방학동신동아타워,"솔밭근린공원, 한강, 불광천, 진관사계곡, 진관사(서울)",지하철,솔밭공원역,수락산역,"불암산, 서울창포원, 백악마루, 송추계곡, 일영허브랜드(숲길정원)"
2,"[접지되는, 속에, 생겼으면, 군데군데, 환관들의묘가, 와보셨으면, 좋겠어용아, 흙...",창경궁대온실,지하철,혜화역,용마산역,"용마산, 초안산, 아차산 어울림정원, 여의도샛강생태공원, 긴고랑계곡",버스,용마폭포공원,구산문화마을.구산성당,"하남 나무고아원, 오동근린공원 유아숲체험장, 오동근린공원, 길동생태공원, 일자산자연공원"
3,"[오염수잔뜩있는곳절대, 굳이, 날씨, 정수처리장, 농구장에, 별게, 편합니다, 산책...",창경궁 명정전,버스,창경궁.서울대학교병원(중),서빙고역1번출구,"용산공원 부분개방부지, 거북선나루터, 홍제폭포, 녹번서공원, 삼각산",지하철,서빙고역,청계산입구역,"여의천, 서울대공원산림욕장, 산기슭 공원, 대모산숲속야생화원, 일원에코파크"
4,"[내용이나, 가이드분, 여기에, 시리우스, 생각보다, 볼게, 관측해보고, 한분, 왔...",창경궁 자생식물학습장,지하철,혜화역,솔밭공원역,"북한산 백운대(우이동), 아차산, 아차산 고구려정, 망우산(서울)",버스,방학동신동아타워,권율장군묘,"송암스페이스센터, 백사실계곡, 수락산, 산들소리"
5,[No keywords],창경궁 홍화문,버스,창경궁.서울대학교병원(중),None,없음,지하철,None,None,없음
6,"[진달래의, 조성돼, 동산에, 활짝, 좋고소소하게, 힐링이구나, 턱없이, 쾌적하고,...",수성동계곡,지하철,독립문역,염창역,150년 수령 느티나무,버스,목동3단지,푸른수목원,"푸른수목원, 무릉도원수목원, 부천자연생태박물관, 진달래동산, 도덕산공원"
7,"[롯데면세점도, 빠져보세요, 작아, 생기고, 정수처리장, 농구장에, 정리가, 필요함...",남산순환나들길,버스,남산3호터널,우면산터널TG(경유),"우면산자연생태공원, 신도림테크노근린공원",지하철,남부터미널역,정부과천청사역,"과천야생화자연학습장, 마루공원, 과천 은행나무길, 청계산맑은숲공원, 백운호수"
8,[No keywords],남산인권숲,지하철,명동역,None,없음,버스,None,None,없음
9,[No keywords],선바위(선암),버스,독립문역.한성과학고(중),None,없음,지하철,None,None,없음


In [23]:
def create_sentence(keywords):
    if not keywords:  # 키워드가 비어있는 경우
        return "특징을 찾을 수 없습니다."
    return f"이 장소는 {', '.join(keywords[::])} 등이 특징입니다."  # 최대 3개의 키워드 사용

# 모든 행의 키워드를 문장으로 변환하여 새 열 추가
result_df['추천음식점소개'] = result_df['모든추천장소키워드'].apply(create_sentence)

In [24]:
result_df

,모든추천장소키워드,추천장소점1,이동수단1,타는곳1,내리는곳1,추천장소2,이동수단2,타는곳2,내리는곳2,추천장소3,추천음식점소개
0,"[운동할수, 서울숲과는, 정비되어, 지나가는, 평화롭고, 좋다, 편합니다, 걷기만,...",서대문안산,지하철,무악재역,북한산보국문역,"북한산국립공원(서울), 월드컵공원 유아숲체험원, 난지천공원, 안양천제방벚꽃길, 홍릉숲",버스,국민대학교앞,상계동주유소,"수락산 당고개지구 공원, 무악산 동봉수대 터, 봉화산(서울), 홍제천인공폭포, 벽운계곡","이 장소는 운동할수, 서울숲과는, 정비되어, 지나가는, 평화롭고, 좋다, 편합니다,..."
1,"[안좋고장난감들이, 사찰이, 생겨, 자전거, 작가의, 국립공원, 센터가, 방문했는데...",인왕산,버스,독립문고가차도(경유),방학동신동아타워,"솔밭근린공원, 한강, 불광천, 진관사계곡, 진관사(서울)",지하철,솔밭공원역,수락산역,"불암산, 서울창포원, 백악마루, 송추계곡, 일영허브랜드(숲길정원)","이 장소는 안좋고장난감들이, 사찰이, 생겨, 자전거, 작가의, 국립공원, 센터가, ..."
2,"[접지되는, 속에, 생겼으면, 군데군데, 환관들의묘가, 와보셨으면, 좋겠어용아, 흙...",창경궁대온실,지하철,혜화역,용마산역,"용마산, 초안산, 아차산 어울림정원, 여의도샛강생태공원, 긴고랑계곡",버스,용마폭포공원,구산문화마을.구산성당,"하남 나무고아원, 오동근린공원 유아숲체험장, 오동근린공원, 길동생태공원, 일자산자연공원","이 장소는 접지되는, 속에, 생겼으면, 군데군데, 환관들의묘가, 와보셨으면, 좋겠어..."
3,"[오염수잔뜩있는곳절대, 굳이, 날씨, 정수처리장, 농구장에, 별게, 편합니다, 산책...",창경궁 명정전,버스,창경궁.서울대학교병원(중),서빙고역1번출구,"용산공원 부분개방부지, 거북선나루터, 홍제폭포, 녹번서공원, 삼각산",지하철,서빙고역,청계산입구역,"여의천, 서울대공원산림욕장, 산기슭 공원, 대모산숲속야생화원, 일원에코파크","이 장소는 오염수잔뜩있는곳절대, 굳이, 날씨, 정수처리장, 농구장에, 별게, 편합니..."
4,"[내용이나, 가이드분, 여기에, 시리우스, 생각보다, 볼게, 관측해보고, 한분, 왔...",창경궁 자생식물학습장,지하철,혜화역,솔밭공원역,"북한산 백운대(우이동), 아차산, 아차산 고구려정, 망우산(서울)",버스,방학동신동아타워,권율장군묘,"송암스페이스센터, 백사실계곡, 수락산, 산들소리","이 장소는 내용이나, 가이드분, 여기에, 시리우스, 생각보다, 볼게, 관측해보고, ..."
5,[No keywords],창경궁 홍화문,버스,창경궁.서울대학교병원(중),None,없음,지하철,None,None,없음,이 장소는 No keywords 등이 특징입니다.
6,"[진달래의, 조성돼, 동산에, 활짝, 좋고소소하게, 힐링이구나, 턱없이, 쾌적하고,...",수성동계곡,지하철,독립문역,염창역,150년 수령 느티나무,버스,목동3단지,푸른수목원,"푸른수목원, 무릉도원수목원, 부천자연생태박물관, 진달래동산, 도덕산공원","이 장소는 진달래의, 조성돼, 동산에, 활짝, 좋고소소하게, 힐링이구나, 턱없이, ..."
7,"[롯데면세점도, 빠져보세요, 작아, 생기고, 정수처리장, 농구장에, 정리가, 필요함...",남산순환나들길,버스,남산3호터널,우면산터널TG(경유),"우면산자연생태공원, 신도림테크노근린공원",지하철,남부터미널역,정부과천청사역,"과천야생화자연학습장, 마루공원, 과천 은행나무길, 청계산맑은숲공원, 백운호수","이 장소는 롯데면세점도, 빠져보세요, 작아, 생기고, 정수처리장, 농구장에, 정리가..."
8,[No keywords],남산인권숲,지하철,명동역,None,없음,버스,None,None,없음,이 장소는 No keywords 등이 특징입니다.
9,[No keywords],선바위(선암),버스,독립문역.한성과학고(중),None,없음,지하철,None,None,없음,이 장소는 No keywords 등이 특징입니다.


In [25]:
import re
from collections import Counter

# ▣ 0. 패턴 · 불용어 정의 -------------------------------------------------
STOPWORDS = {
    '곳', '맛', '일', '개', '어쩌구', '집', '곳이', '곳입니다'
}

MENU_KEY   = r"(짜장|짬뽕|탕수육|파스타|스테이크|갈비|돈까스|냉면|볶음밥)$"
TASTE_KEY  = r"(맛있음|존맛|존맛탱|훌륭|완벽|꿀맛|강추|재밌음|만족)$"

# ▣ 1. 혼합 토큰 분리 ---------------------------------------------------
def split_mixed_token(tok: str) -> list[str]:
    """
    '진갈비어쩌구인데맛있음' → ['진갈비어쩌구인데', '맛있음']
    """
    m = re.search(TASTE_KEY, tok)
    if m:
        taste = m.group()
        dish  = re.sub(TASTE_KEY + r".*", "", tok)          # 감상어 뒤쪽 삭제
        return [dish] if not dish else [dish, taste]
    return [tok]

# ▣ 2. 전처리 -----------------------------------------------------------
def clean_tokens(tokens: list[str]) -> list[str]:
    """
    - 공백·불용어·특수기호·1~2자 일반어 제거
    - '개맛있음' ⇒ '맛있음'  (강세 접두사 '개' 제거)
    """
    out = []
    for t in tokens:
        t = t.strip()
        if not t or t in STOPWORDS:
            continue

        # 접두사 '개' 제거
        if t.startswith('개') and len(t) > 2:
            t = t[1:]

        # 특수기호·짧은 토큰 제거
        if re.fullmatch(r"[^\w가-힣]+", t):
            continue
        if len(t) <= 2 and re.fullmatch(r"[A-Za-z가-힣]+", t):
            continue

        out.append(t)
    return out

# ▣ 3. 메뉴명 정규화 ----------------------------------------------------
def normalize_menu(word: str) -> str:
    """
    '진갈비어쩌구인데' → '진갈비'
    '짜장면은'         → '짜장면'
    """
    m = re.search(MENU_KEY, word)
    core = word[:m.end()] if m else word            # 메뉴키워드까지 잘라냄
    core = re.sub(r"(은|는|이|가|을|를)$", "", core) # 끝 조사 제거
    return core

# ▣ 4. 음식이 없을 때 “주 대상” 잡기 ------------------------------------
def pick_main_keyword(tokens: list[str],
                      reserved: set[str],
                      fallback: str = "이곳") -> str:
    """
    ① 음식(Menu) 있으면 그것 사용
    ② 없으면 남은 토큰 중 명사형(2글자↑) 빈도 1위
    ③ 없으면 fallback
    """
    food = [t for t in tokens if re.search(MENU_KEY, t)]
    if food:
        return normalize_menu(Counter(food).most_common(1)[0][0])

    candidates = [t for t in tokens if t not in reserved]
    noun_like  = [t for t in candidates if re.fullmatch(r"[가-힣A-Za-z]{2,}", t)]
    if noun_like:
        return Counter(noun_like).most_common(1)[0][0]

    return fallback

# ▣ 5. 메인 변환 함수 ---------------------------------------------------
def keywords_to_sentence(keyword_blob) -> str:
    """
    keyword_blob : 쉼표로 구분된 str  또는  키워드 list
    return       : 자연스러운 한 문장
    """
    # 5-1) 토큰화
    if isinstance(keyword_blob, str):
        blob = re.sub(r"^이\s+장소는\s*", "", keyword_blob)  # 프리픽스 제거
        raw_tokens = [tok.strip() for tok in blob.split(',')]
    else:
        raw_tokens = list(keyword_blob)

    # 5-2) 혼합 토큰 분리 + 클린
    tokens = []
    for t in raw_tokens:
        tokens.extend(split_mixed_token(t))
    tokens = clean_tokens(tokens)

    # 5-3) 범주별 후보 추출
    dishes   = [t for t in tokens if re.search(MENU_KEY, t)]
    people   = [t for t in tokens if t.endswith(('와', '랑'))]               # 외할머니와 …
    tastes   = [t for t in tokens if re.fullmatch(TASTE_KEY, t)]
    ambience = [t for t in tokens if t in ('깔끔한', '분위기로', '친절하고', '편했답니다')]
    places   = [t for t in tokens if t.endswith(('타운', '동', '차이나타운', '식당'))]

    pick = lambda lst: Counter(lst).most_common(1)[0][0] if lst else ""

    reserved = set(sum([dishes, people, tastes, ambience, places], []))
    main_kw  = pick_main_keyword(tokens, reserved)

    who   = pick(people)
    taste = pick(tastes) or "괜찮았습니다"
    vibe  = pick(ambience)
    where = pick(places)

    # 5-4) 템플릿 조립
    sentence  = f"{where + '에서 ' if where else ''}"
    sentence += f"{who + ' 함께 ' if who else ''}{main_kw}을(를) "
    sentence += f"경험했는데 {taste}"
    if vibe:
        sentence += f"({vibe} 분위기)"

    return sentence



In [26]:
result_df["한줄소개"] = result_df["모든추천장소키워드"].apply(keywords_to_sentence)

In [27]:
# result_df.to_excel('result.xlsx', index=False, sheet_name='ResultData')

In [28]:
result_df

,모든추천장소키워드,추천장소점1,이동수단1,타는곳1,내리는곳1,추천장소2,이동수단2,타는곳2,내리는곳2,추천장소3,추천음식점소개,한줄소개
0,"[운동할수, 서울숲과는, 정비되어, 지나가는, 평화롭고, 좋다, 편합니다, 걷기만,...",서대문안산,지하철,무악재역,북한산보국문역,"북한산국립공원(서울), 월드컵공원 유아숲체험원, 난지천공원, 안양천제방벚꽃길, 홍릉숲",버스,국민대학교앞,상계동주유소,"수락산 당고개지구 공원, 무악산 동봉수대 터, 봉화산(서울), 홍제천인공폭포, 벽운계곡","이 장소는 운동할수, 서울숲과는, 정비되어, 지나가는, 평화롭고, 좋다, 편합니다,...",홍제동에서 나무와 함께 운동할수을(를) 경험했는데 괜찮았습니다
1,"[안좋고장난감들이, 사찰이, 생겨, 자전거, 작가의, 국립공원, 센터가, 방문했는데...",인왕산,버스,독립문고가차도(경유),방학동신동아타워,"솔밭근린공원, 한강, 불광천, 진관사계곡, 진관사(서울)",지하철,솔밭공원역,수락산역,"불암산, 서울창포원, 백악마루, 송추계곡, 일영허브랜드(숲길정원)","이 장소는 안좋고장난감들이, 사찰이, 생겨, 자전거, 작가의, 국립공원, 센터가, ...",주방쪽이랑 함께 안좋고장난감들이을(를) 경험했는데 괜찮았습니다(깔끔한 분위기)
2,"[접지되는, 속에, 생겼으면, 군데군데, 환관들의묘가, 와보셨으면, 좋겠어용아, 흙...",창경궁대온실,지하철,혜화역,용마산역,"용마산, 초안산, 아차산 어울림정원, 여의도샛강생태공원, 긴고랑계곡",버스,용마폭포공원,구산문화마을.구산성당,"하남 나무고아원, 오동근린공원 유아숲체험장, 오동근린공원, 길동생태공원, 일자산자연공원","이 장소는 접지되는, 속에, 생겼으면, 군데군데, 환관들의묘가, 와보셨으면, 좋겠어...",걷기운동에서 흙이랑 함께 접지되는을(를) 경험했는데 괜찮았습니다
3,"[오염수잔뜩있는곳절대, 굳이, 날씨, 정수처리장, 농구장에, 별게, 편합니다, 산책...",창경궁 명정전,버스,창경궁.서울대학교병원(중),서빙고역1번출구,"용산공원 부분개방부지, 거북선나루터, 홍제폭포, 녹번서공원, 삼각산",지하철,서빙고역,청계산입구역,"여의천, 서울대공원산림욕장, 산기슭 공원, 대모산숲속야생화원, 일원에코파크","이 장소는 오염수잔뜩있는곳절대, 굳이, 날씨, 정수처리장, 농구장에, 별게, 편합니...",오염수와 함께 오염수잔뜩있는곳절대을(를) 경험했는데 괜찮았습니다
4,"[내용이나, 가이드분, 여기에, 시리우스, 생각보다, 볼게, 관측해보고, 한분, 왔...",창경궁 자생식물학습장,지하철,혜화역,솔밭공원역,"북한산 백운대(우이동), 아차산, 아차산 고구려정, 망우산(서울)",버스,방학동신동아타워,권율장군묘,"송암스페이스센터, 백사실계곡, 수락산, 산들소리","이 장소는 내용이나, 가이드분, 여기에, 시리우스, 생각보다, 볼게, 관측해보고, ...",내용이나을(를) 경험했는데 괜찮았습니다
5,[No keywords],창경궁 홍화문,버스,창경궁.서울대학교병원(중),None,없음,지하철,None,None,없음,이 장소는 No keywords 등이 특징입니다.,이곳을(를) 경험했는데 괜찮았습니다
6,"[진달래의, 조성돼, 동산에, 활짝, 좋고소소하게, 힐링이구나, 턱없이, 쾌적하고,...",수성동계곡,지하철,독립문역,염창역,150년 수령 느티나무,버스,목동3단지,푸른수목원,"푸른수목원, 무릉도원수목원, 부천자연생태박물관, 진달래동산, 도덕산공원","이 장소는 진달래의, 조성돼, 동산에, 활짝, 좋고소소하게, 힐링이구나, 턱없이, ...",아기랑 함께 진달래의을(를) 경험했는데 괜찮았습니다
7,"[롯데면세점도, 빠져보세요, 작아, 생기고, 정수처리장, 농구장에, 정리가, 필요함...",남산순환나들길,버스,남산3호터널,우면산터널TG(경유),"우면산자연생태공원, 신도림테크노근린공원",지하철,남부터미널역,정부과천청사역,"과천야생화자연학습장, 마루공원, 과천 은행나무길, 청계산맑은숲공원, 백운호수","이 장소는 롯데면세점도, 빠져보세요, 작아, 생기고, 정수처리장, 농구장에, 정리가...",롯데면세점도을(를) 경험했는데 괜찮았습니다(친절하고 분위기)
8,[No keywords],남산인권숲,지하철,명동역,None,없음,버스,None,None,없음,이 장소는 No keywords 등이 특징입니다.,이곳을(를) 경험했는데 괜찮았습니다
9,[No keywords],선바위(선암),버스,독립문역.한성과학고(중),None,없음,지하철,None,None,없음,이 장소는 No keywords 등이 특징입니다.,이곳을(를) 경험했는데 괜찮았습니다
